# Build Targets — Contract Enforcement

**Purpose**: Construct the ground truth target table that enforces the prediction contract.

**What this notebook does**:
- Creates a player × GW target frame with explicit labels
- No features, no joins to prior GWs — just outcomes for GW *t*

**What this notebook does NOT do**:
- No feature engineering
- No modeling
- No train/test splits (that comes later)

**Contract Reference**: `docs/PREDICTION_CONTRACT.md`

---

In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

# Paths
DB_PATH = Path("../storage/fpl_2025_26.sqlite")
OUTPUT_PATH = Path("../storage/datasets")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Connect
conn = sqlite3.connect(DB_PATH)
print(f"Connected to: {DB_PATH}")

Connected to: ../storage/fpl_2025_26.sqlite


## 1. Target Definitions

From the prediction contract:

| Target | Definition | Type |
|--------|------------|------|
| `y_points` | `total_points` in GW *t* | Regression |
| `y_play` | 1 if `minutes > 0` in GW *t* | Binary |
| `y_60` | 1 if `minutes ≥ 60` in GW *t* | Binary |
| `y_haul` | 1 if `total_points ≥ 10` in GW *t* | Binary |

Each row represents:
> "What we would have predicted before GW *t*, and what actually happened in GW *t*."

In [2]:
# Load gameweeks data (outcomes only)
query = """
SELECT 
    element_id AS player_id,
    round AS gw,
    total_points,
    minutes
FROM gameweeks
ORDER BY element_id, round
"""

gw_df = pd.read_sql(query, conn)
print(f"Loaded {len(gw_df):,} player × GW rows")
print(f"GW range: {gw_df['gw'].min()} to {gw_df['gw'].max()}")
print(f"Unique players: {gw_df['player_id'].nunique():,}")

Loaded 16,559 player × GW rows
GW range: 1 to 22
Unique players: 799


## 2. Construct Target Columns

In [3]:
# Build target frame
targets = pd.DataFrame({
    'player_id': gw_df['player_id'],
    'gw': gw_df['gw'],
    'y_points': gw_df['total_points'],
    'y_play': (gw_df['minutes'] > 0).astype(int),
    'y_60': (gw_df['minutes'] >= 60).astype(int),
    'y_haul': (gw_df['total_points'] >= 10).astype(int),
})

print("Target frame created:")
print(targets.head(10))

Target frame created:
   player_id  gw  y_points  y_play  y_60  y_haul
0          1   1        10       1     1       1
1          1   2         6       1     1       0
2          1   3         2       1     1       0
3          1   4         6       1     1       0
4          1   5         2       1     1       0
5          1   6         2       1     1       0
6          1   7         6       1     1       0
7          1   8         6       1     1       0
8          1   9         6       1     1       0
9          1  10         6       1     1       0


## 3. Target Distribution Summary

In [4]:
print("=" * 50)
print("TARGET DISTRIBUTIONS")
print("=" * 50)

# y_points (regression)
print(f"\ny_points (regression target):")
print(f"  Mean:   {targets['y_points'].mean():.2f}")
print(f"  Median: {targets['y_points'].median():.1f}")
print(f"  Std:    {targets['y_points'].std():.2f}")
print(f"  Min:    {targets['y_points'].min()}")
print(f"  Max:    {targets['y_points'].max()}")

# Binary targets
for col in ['y_play', 'y_60', 'y_haul']:
    rate = targets[col].mean()
    count = targets[col].sum()
    print(f"\n{col}:")
    print(f"  Positive rate: {rate:.1%} ({count:,} / {len(targets):,})")

TARGET DISTRIBUTIONS

y_points (regression target):
  Mean:   1.21
  Median: 0.0
  Std:    2.41
  Min:    -3
  Max:    24

y_play:
  Positive rate: 40.3% (6,666 / 16,559)

y_60:
  Positive rate: 27.4% (4,536 / 16,559)

y_haul:
  Positive rate: 1.8% (302 / 16,559)


## 4. Validation Checks

In [5]:
print("=" * 50)
print("VALIDATION CHECKS")
print("=" * 50)

# Check 1: No missing values
missing = targets.isnull().sum().sum()
print(f"\n✓ Missing values: {missing}")
assert missing == 0, "Missing values found!"

# Check 2: y_60 implies y_play
invalid_60 = ((targets['y_60'] == 1) & (targets['y_play'] == 0)).sum()
print(f"✓ y_60=1 but y_play=0: {invalid_60} (should be 0)")
assert invalid_60 == 0, "y_60 can't be 1 if y_play is 0!"

# Check 3: y_haul implies y_play (can't haul without playing)
invalid_haul = ((targets['y_haul'] == 1) & (targets['y_play'] == 0)).sum()
print(f"✓ y_haul=1 but y_play=0: {invalid_haul} (should be 0)")
assert invalid_haul == 0, "y_haul can't be 1 if y_play is 0!"

# Check 4: Unique player × GW combinations
duplicates = targets.duplicated(subset=['player_id', 'gw']).sum()
print(f"✓ Duplicate player×GW rows: {duplicates} (should be 0)")
assert duplicates == 0, "Duplicate player×GW combinations found!"

# Check 5: GW range is contiguous
gw_range = set(range(targets['gw'].min(), targets['gw'].max() + 1))
actual_gws = set(targets['gw'].unique())
missing_gws = gw_range - actual_gws
print(f"✓ Missing GWs in range: {missing_gws if missing_gws else 'None'}")

print("\n" + "=" * 50)
print("ALL CHECKS PASSED ✅")
print("=" * 50)

VALIDATION CHECKS

✓ Missing values: 0
✓ y_60=1 but y_play=0: 0 (should be 0)
✓ y_haul=1 but y_play=0: 0 (should be 0)
✓ Duplicate player×GW rows: 0 (should be 0)
✓ Missing GWs in range: None

ALL CHECKS PASSED ✅


## 5. Temporal Boundary Verification

> **Contract requirement**: Each row corresponds to a prediction made before GW *t*, with targets from GW *t* outcomes.

This table contains **only outcomes**. Features (from GWs ≤ *t−1*) will be joined later.

In [6]:
# Show sample: "For GW 10, these are the outcomes we're predicting"
sample_gw = 10
sample = targets[targets['gw'] == sample_gw].head(10)

print(f"Sample targets for GW {sample_gw}:")
print(f"(These outcomes would be predicted using data from GW 1-{sample_gw-1})")
print()
print(sample.to_string(index=False))

Sample targets for GW 10:
(These outcomes would be predicted using data from GW 1-9)

 player_id  gw  y_points  y_play  y_60  y_haul
         1  10         6       1     1       0
         2  10         0       0     0       0
         3  10         0       0     0       0
         4  10         0       0     0       0
         5  10        12       1     1       1
         6  10         6       1     1       0
         7  10         6       1     1       0
         8  10         6       1     1       0
         9  10         0       0     0       0
        10  10         1       1     0       0


## 6. Save Target Frame

In [7]:
# Save to CSV
output_file = OUTPUT_PATH / "targets.csv"
targets.to_csv(output_file, index=False)

print(f"Saved target frame to: {output_file}")
print(f"Shape: {targets.shape}")
print(f"Columns: {list(targets.columns)}")

Saved target frame to: ../storage/datasets/targets.csv
Shape: (16559, 6)
Columns: ['player_id', 'gw', 'y_points', 'y_play', 'y_60', 'y_haul']


## 7. Summary

**Stage 2 — Target Construction (Contract Enforcement)**

| Artifact | Description |
|----------|-------------|
| `storage/datasets/targets.csv` | Ground truth target table |

### Schema

| Column | Type | Description |
|--------|------|-------------|
| `player_id` | int | Player identifier (FK to players.id) |
| `gw` | int | Target gameweek (*t*) |
| `y_points` | int | Points scored in GW *t* (regression target) |
| `y_play` | int {0,1} | Binary indicator: 1 if minutes > 0 |
| `y_60` | int {0,1} | Binary indicator: 1 if minutes ≥ 60 |
| `y_haul` | int {0,1} | Binary indicator: 1 if points ≥ 10 |

### Coverage Note

Rows exist only for player×GW pairs where the player was in the FPL game pool for GW *t*.  
Players added mid-season have no rows for earlier GWs; players removed have no rows for later GWs.

### Next Step

Build features using data from GWs ≤ *t−1* and join to this target frame.

In [8]:
# Cleanup
conn.close()
print("Done.")

Done.
